In [2]:
import os
ES_SERVER_HOST = os.getenv("ES_SERVER_HOST", "3.35.110.161")
ES_SERVER_PORT = int(os.getenv("ES_SERVER_PORT", "9200"))
ES_SERVER_USERNAME = os.getenv("ES_SERVER_USERNAME", "elastic")
ES_SERVER_PASSWORD = os.getenv("ES_SERVER_PASSWORD", "snomed")

from elasticsearch import Elasticsearch

# ES 8.x 문법 (기본 권장)
es = Elasticsearch(
    f"http://{ES_SERVER_HOST}:{ES_SERVER_PORT}",
    basic_auth=(ES_SERVER_USERNAME, ES_SERVER_PASSWORD),
    request_timeout=60,
)

In [4]:
# 연결 확인
info = es.info()
print("Cluster name:", info.get('cluster_name'))
print("Version:", info.get('version', {}).get('number'))

# 인덱스 확인
indices = sorted(es.indices.get(index="*").keys())
print(f"\n총 인덱스 수: {len(indices)}")

Cluster name: docker-cluster
Version: 9.0.0

총 인덱스 수: 34


In [6]:
rows = es.cat.indices(format="json", s="index", bytes="mb")
df = pd.DataFrame(rows)[["index", "docs.count", "store.size"]]
df.rename(columns={"index":"index_name","docs.count":"docs","store.size":"store_mb"}, inplace=True)
df

,index_name,docs,store_mb
0,concept-condition,268040,70
1,concept-condition-device,1,0
2,concept-condition-meas,8,0
3,concept-condition_status,22,0
4,concept-cost,51,0
5,concept-currency,180,0
6,concept-device,237173,56
7,concept-drug,4772484,1292
8,concept-episode,18,0
9,concept-ethnicity,2,0


In [1]:
import pandas as pd

df = pd.read_csv('/Users/rose/Desktop/omop-mapper/data/CONCEPT.csv', sep='\t')
df.head()

/var/folders/km/l4y95bgj6lv844p6y3h17gcc0000gn/T/ipykernel_55773/2066303231.py:3: DtypeWarning: Columns (5,6,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/Users/rose/Desktop/omop-mapper/data/CONCEPT.csv', sep='\t')


,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
0,45756805,Pediatric Cardiology,Provider,ABMS,Physician Specialty,S,OMOP4821938,19700101,20991231,NaN
1,45756804,Pediatric Anesthesiology,Provider,ABMS,Physician Specialty,S,OMOP4821939,19700101,20991231,NaN
2,45756803,Pathology-Anatomic / Pathology-Clinical,Provider,ABMS,Physician Specialty,S,OMOP4821940,19700101,20991231,NaN
3,45756802,Pathology - Pediatric,Provider,ABMS,Physician Specialty,S,OMOP4821941,19700101,20991231,NaN
4,45756801,Pathology - Molecular Genetic,Provider,ABMS,Physician Specialty,S,OMOP4821942,19700101,20991231,NaN


In [7]:
import time
from elasticsearch import Elasticsearch
from elasticsearch.exceptions import AuthorizationException

# --- Elasticsearch 연결 정보 수정 ---
ES_HOST = "http://3.35.110.161:9200"
ES_USER = "elastic"
ES_PASSWORD = "snomed"
# ------------------------------------

try:
    # Elasticsearch 클라이언트 생성
    es = Elasticsearch(
        [ES_HOST],
        basic_auth=(ES_USER, ES_PASSWORD)
    )

    # 1. ML 업그레이드 모드 활성화
    print("Step 1: Enabling ML upgrade mode...")
    es.ml.set_upgrade_mode(enabled=True)
    print("Successfully enabled. Waiting for 5 seconds...")

    # 안정적인 상태 전환을 위해 잠시 대기
    time.sleep(5)

    # 2. ML 업그레이드 모드 비활성화 (상태 리셋)
    print("Step 2: Disabling ML upgrade mode to reset state...")
    es.ml.set_upgrade_mode(enabled=False)
    print("Successfully disabled. ML system state has been refreshed!")

except AuthorizationException as e:
    print(f"Authentication Error: Please check your username and password. Details: {e.error}")
except Exception as e:
    print(f"An error occurred: {e}")

Step 1: Enabling ML upgrade mode...
Successfully enabled. Waiting for 5 seconds...
Step 2: Disabling ML upgrade mode to reset state...
Successfully disabled. ML system state has been refreshed!


In [9]:
from elasticsearch import Elasticsearch

# --- Elasticsearch 연결 정보 수정 ---
ES_HOST = "http://3.35.110.161:9200"
ES_USER = "elastic"
ES_PASSWORD = "snomed"
# ------------------------------------

try:
    # Elasticsearch 클라이언트 생성
    es = Elasticsearch(
        [ES_HOST],
        basic_auth=(ES_USER, ES_PASSWORD),
        request_timeout=30
    )

    # 클러스터의 모든 노드 정보 가져오기
    # 'settings'와 'roles' 정보만 필터링하여 요청
    nodes_info = es.nodes.info(metric=["settings", "roles"])

    print("--- Cluster Node Roles ---")

    ml_node_found = False
    for node_id, info in nodes_info["nodes"].items():
        node_name = info["name"]
        roles = info["roles"]
        
        print(f"Node Name: {node_name}")
        print(f"  - Roles: {roles}")
        
        if "ml" in roles:
            ml_node_found = True
            print("  - ✅ This node is an ML node.")
        else:
            print("  - ❌ This node is NOT an ML node.")
        print("-" * 20)

    if not ml_node_found:
        print("\n[CRITICAL] No ML nodes found in the cluster!")
        print("Please edit 'elasticsearch.yml' on at least one node to add the 'ml' role and restart it.")
    else:
        print("\n[INFO] ML node(s) found in the cluster.")

except Exception as e:
    print(f"An error occurred: {e}")

--- Cluster Node Roles ---

[CRITICAL] No ML nodes found in the cluster!
Please edit 'elasticsearch.yml' on at least one node to add the 'ml' role and restart it.
